<a href="https://colab.research.google.com/github/01PrathamS/Backprop/blob/main/micrograd_karpathy_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import math
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [103]:
x1 = torch.Tensor([2.0]).double()           ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()           ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()          ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()           ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [174]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out =  Value(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out =  Value(self.data * other.data, (self, other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, other):
    out = Value(self.data**other, (self, ), f'**{other}')
    def _backward():
      self.grad += other * (self.data ** (other-1))
    out._backward = _backward
    return out

  def __rmul__(self, other):
    return self * other

  def __truediv__(self, other):
    return self * other**-1

  def __sub__(self, other):
    return self + (-other)

  def __neg__(self):
    return self * -1

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)

    out = Value(t, (self, ), 'tanh')
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    def _backward():
      self.grad = out.data * out.grad
    out._backward = _backward
    return out


  def backward(self):
    ## topological graph
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [175]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1, 1))

  def __call__(self, x):
    ## w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w +  [self.b]

x = [2.0, 3.0]
n = Neuron(2)
n(x)
n.parameters()


[Value(data=-0.6979620815976473),
 Value(data=0.5643701321781274),
 Value(data=0.5027477915945704)]

In [176]:
class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]


# x = [2.0, 3.0]
# n = Layer(2, 3)
# n(x)
# print(len(n.parameters()))

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 2])
n(x)
len(n.parameters())

## layer wise parameters
## first layer parameters --> 3 * 4 = 12 weights + 4 bias ==> 16 params
## second layer parameters --> 4 * 4 = 16 weights + 4 bias ==> 20 parms
## third layer parameterse --> 4 * 2 = 8 weights + 2 bias ==> 10 param // total 46 parameters for this network

46

In [201]:
# x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
# n(x)

In [161]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [n(x) for x in xs]
ypred


[Value(data=-0.1326105791523551),
 Value(data=-0.40039456639263477),
 Value(data=-0.2869555016481003),
 Value(data=0.13020692881563262)]

In [197]:
ypred = [n(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=3.1164906329606588)

In [195]:
loss.backward()

In [192]:
n.layers[0].neurons[0].w[0].grad

-0.8819552285190739

In [189]:
n.layers[0].neurons[0].w[0].data

0.7896563439008284

In [196]:
for p in n.parameters():
  p.data += -0.01 * p.grad

In [202]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

In [206]:
for k in range(10):
  ## forward pass

  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  ## zero grad
  for p in n.parameters():
    p.grad = 0.0

  ## backward loss
  loss.backward()

  ## update
  for p in n.parameters():
    p.data += -0.05 * p.grad

  print(k, loss.data)

0 0.4261320329350683
1 0.310995744917679
2 0.2372953842082719
3 0.18823154473388748
4 0.15411236397237527
5 0.12941897267880012
6 0.11092212500391949
7 0.09665830656353526
8 0.08538594207758361
9 0.07629088776258451


In [207]:
ypred

[Value(data=0.8963429701839797),
 Value(data=-0.9498735467780612),
 Value(data=-0.8261027264926206),
 Value(data=0.8189111132995908)]